# Radiative transfer model of HD209458b

Main reference: https://www.aanda.org/articles/aa/full_html/2010/12/aa13396-09/aa13396-09.html

## Question 1

We reformulate the FTE using pressure as the coordinate. We have the FTE

$$ \mu \frac{dI}{dz} = \rho \kappa (B(T(z)) - I) $$

Using the chain rule, we have

$$ \mu \frac{dI}{dP} \frac{dP}{dz} = \rho \kappa (B(T(z)) - I) $$

Substituting the equation for hydrostatic equilibrium $\frac{dP}{dz} = -g \rho$, we get

$$ -\mu \frac{dI}{dP} = \frac{\kappa}{g} (B(T(P)) - I) $$

Working in the P-coordinate has a few advantages:
1. No need to know the equation of state of the gas
2. Gas elements stays glued to their value of P, even if the temperature $T(P)$ changes

## Question 2 & 3

We set up a logarithmic grid $\{P_i\}_{i=0}^{N-1}$ and rewrite the numerical integration.

In [1]:
import numpy as np

# some setup variables

N = 100
B = 1

In [3]:
# setup logarithmic grid
# runs from high p (surface) to low p (top)
# units are in bar

p = np.logspace(-5, 0, N) 
p = p[::-1]
p

array([1.00000000e+00, 8.90215085e-01, 7.92482898e-01, 7.05480231e-01,
       6.28029144e-01, 5.59081018e-01, 4.97702356e-01, 4.43062146e-01,
       3.94420606e-01, 3.51119173e-01, 3.12571585e-01, 2.78255940e-01,
       2.47707636e-01, 2.20513074e-01, 1.96304065e-01, 1.74752840e-01,
       1.55567614e-01, 1.38488637e-01, 1.23284674e-01, 1.09749877e-01,
       9.77009957e-02, 8.69749003e-02, 7.74263683e-02, 6.89261210e-02,
       6.13590727e-02, 5.46227722e-02, 4.86260158e-02, 4.32876128e-02,
       3.85352859e-02, 3.43046929e-02, 3.05385551e-02, 2.71858824e-02,
       2.42012826e-02, 2.15443469e-02, 1.91791026e-02, 1.70735265e-02,
       1.51991108e-02, 1.35304777e-02, 1.20450354e-02, 1.07226722e-02,
       9.54548457e-03, 8.49753436e-03, 7.56463328e-03, 6.73415066e-03,
       5.99484250e-03, 5.33669923e-03, 4.75081016e-03, 4.22924287e-03,
       3.76493581e-03, 3.35160265e-03, 2.98364724e-03, 2.65608778e-03,
       2.36448941e-03, 2.10490414e-03, 1.87381742e-03, 1.66810054e-03,
      

## Question 4

To include the irradiation of the atmosphere from the star (with flux $F_v^*$, we use the superposition principle. As the flux enters the atmosphere with an angle $i$ with respect to the z-axis, the equation for the attenuation of the flux is

$$ \frac{dF_v^{*}(P)}{dP} = - \frac{\kappa_v}{g \cos(i)} F_v^{*}(P) $$

## Question 5

$F^* = \int_0^\infty F^*_\nu \,d\nu$, the frequency-integrated stellar flux, at the top of the atmosphere is

$$ F^*(P = P_{min}) = \frac{L_*}{4\pi d^2} $$

where $L_*$ is the frequency-integrated luminosity of the star.

## Question 6 and 7

Now, we would like to be able to compute $F^*(P_i)$ and $J(P_i)$ at every grid point $i$. By assuming that the opacity for the stellar photons $\kappa_*$ to be different for the atmospheric thermal photons $\kappa_{th}$, we get the following relation for the atmospheric temperature

$$ \frac{\sigma_{sb}}{\pi} T(P)^4 = J(P) + \frac{\kappa_*}{4\pi\kappa_{th}}F^*(P) $$

To create the stellar radiation field, integrate $F_*$ downward under angle $i$. The ODE is:

$$ \frac{dF_*(P)}{dP} = -\frac{\kappa_*}{g \cos(i)} F_*(P)$$

where $\kappa_* = 4 \times 10^{-3} cm^2/g = 4 \times 10^{-4} m^2/kg$ and $g = 9.66 m/s^2$. Loop:
$$ \Delta\tau = \frac{\kappa_*}{g\cos(i)} |\Delta P|$$
$$ F_*[i-1] = F_*[i] * \exp(-\Delta\tau) $$

And then $J_* = \frac{F_*}{4\pi}$, i.e. the mean intensity of the stellar light.

In [7]:
# default constants in SI units

kappaStar = 4 * 1e-4
g = 9.66
lstar = 3.828 * 1e26
d = 150 * 1e9

def computeF(theta):
    """
    compute irradiation given inclination angle theta.
    """
    Fstar = np.zeros(N)
    Fstar[-1] = lstar/(4 * np.pi * d**2)
    
    for i in range(len(p)-1, -1, -1): # start from the top
        deltaP = p[i-1] - p[i]
        tau = kappaStar/g * 1/np.cos(theta) * deltaP
        Fstar[i-1] = Fstar[i] * np.exp(-tau)
        
    return Fstar

Fstar = computeF(0) # test it for zenith
Jstar = Fstar/(4 * np.pi)
Jstar

array([107.73373087, 107.73422062, 107.73465661, 107.73504474,
       107.73539025, 107.73569784, 107.73597165, 107.73621541,
       107.73643241, 107.73662558, 107.73679755, 107.73695063,
       107.73708691, 107.73720823, 107.73731623, 107.73741238,
       107.73749797, 107.73757416, 107.73764199, 107.73770237,
       107.73775612, 107.73780397, 107.73784657, 107.73788449,
       107.73791825, 107.7379483 , 107.73797505, 107.73799887,
       107.73802007, 107.73803894, 107.73805575, 107.7380707 ,
       107.73808402, 107.73809587, 107.73810642, 107.73811582,
       107.73812418, 107.73813162, 107.73813825, 107.73814415,
       107.7381494 , 107.73815408, 107.73815824, 107.73816194,
       107.73816524, 107.73816818, 107.73817079, 107.73817312,
       107.73817519, 107.73817703, 107.73817867, 107.73818013,
       107.73818144, 107.73818259, 107.73818362, 107.73818454,
       107.73818536, 107.73818609, 107.73818673, 107.73818731,
       107.73818782, 107.73818828, 107.73818869, 107.73

In [9]:
# now compute the temperature at all grid points

# more parameters for HD209458b
kappaThermal = 1e-3
sb = 5.67 * 1e-8
Temperature = np.zeros(N)

for i in range(len(p)):
    temp = (Jstar[i] + kappaStar/kappaThermal * 0.25 * 1/np.pi * Fstar[i]) * np.pi/sb
    temp = temp**(1/4)
    Temperature[i] = temp
Temperature

array([302.35115874, 302.35150236, 302.35180826, 302.35208057,
       302.35232299, 302.35253879, 302.3527309 , 302.35290192,
       302.35305417, 302.3531897 , 302.35331035, 302.35341776,
       302.35351337, 302.35359849, 302.35367426, 302.35374172,
       302.35380177, 302.35385522, 302.35390281, 302.35394517,
       302.35398289, 302.35401646, 302.35404634, 302.35407295,
       302.35409663, 302.35411772, 302.35413649, 302.3541532 ,
       302.35416807, 302.35418131, 302.3541931 , 302.3542036 ,
       302.35421294, 302.35422125, 302.35422866, 302.35423525,
       302.35424111, 302.35424634, 302.35425099, 302.35425512,
       302.35425881, 302.35426209, 302.35426501, 302.35426761,
       302.35426992, 302.35427198, 302.35427382, 302.35427545,
       302.3542769 , 302.3542782 , 302.35427935, 302.35428037,
       302.35428129, 302.3542821 , 302.35428282, 302.35428347,
       302.35428404, 302.35428455, 302.354285  , 302.35428541,
       302.35428577, 302.35428609, 302.35428637, 302.35

## Question 8

Now we use lambda iteration to compute the temperature structure of the atmosphere $T_i = T(P_i)$.

* Start with initial guess of $T(P)$, e.g. $T(P)$ = 2000 K.
* Loop iteration:
    - Compute $B(T) = \frac{\sigma_*}{\pi} * T^4$
    - `B[:] = (sigma_b/pi) * T[:]**4`
    - Source function: (take albedo = 0)
    - `S[:] = B[:]`

Moments of intensity
$$ J = \frac{1}{4\pi} \oint I(\Omega) \,d\Omega = \frac{1}{2} \int_{-1}^{+1} I_{\mu}\,d\mu$$

mean intensity = 0th moment (scalar), 1st moment (vector), 2nd moment (rank 2 tensor).
$$ \frac{1}{4\pi} \oint \vec{n} \cdot \vec{\nabla} I \,d\Omega = \frac{1}{4\pi} \oint \alpha (S - I) \,d\Omega$$
$$ \frac{1}{4\pi}\vec{\nabla} \cdot \oint \vec{n}  I \,d\Omega = \alpha S - \frac{1}{4\pi} \alpha \oint{I \,d\Omega} = \alpha (S - T)$$
$$ \frac{1}{4\pi} \nabla \cdot \oint \vec{n} \vec{n} I \,d\Omega = - \frac{\alpha}{4\pi} \oint I \vec{n} \,d\Omega$$